In [ ]:
import json
import re
import os
from typing import Dict, Any

# --- New Imports for Modern Agent and Tools ---
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import textstat # Library for readability scores

# Load environment variables from .env file
load_dotenv()

# ==============================================================================
#  TOOL LOGIC FUNCTIONS (Separated for clarity)
# ==============================================================================

def check_length_and_structure(text: str) -> str:
    """
    Analyzes content length and structure (word count, paragraphs).
    This is fast and rule-based, perfect for a custom tool.
    Returns a JSON string with the analysis.
    """
    words = text.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    issues = []

    if word_count < 15:
        issues.append(f"Content is very short with only {word_count} words.")
    if word_count > 150 and len(paragraphs) == 1:
        issues.append("Long content is not broken into paragraphs, making it hard to read.")
    if len(sentences) < 2 and word_count > 20:
        issues.append("Content consists of a single long sentence; consider breaking it up.")

    return json.dumps({"word_count": word_count, "sentences": len(sentences), "paragraphs": len(paragraphs), "issues": issues})

def check_grammar_with_llm(text: str) -> str:
    """
    Uses a powerful LLM to check for grammar and spelling errors.
    This is more accurate than a fixed list of regex patterns.
    Returns a JSON string listing the errors found.
    """
    print("🧠 Using LLM to check grammar...")
    checker_llm = ChatOpenAI(model="gpt-4o", temperature=0)
    prompt = f"""
    You are an expert proofreader. Analyze the following text for grammar and spelling mistakes.
    Respond with a JSON object containing one key, "errors", which is a list of strings.
    Each string should describe a specific error you found. If no errors, return an empty list.
    Text: "{text}"
    """
    response = checker_llm.invoke(prompt)
    return response.content

def check_readability_with_textstat(text: str) -> str:
    """
    Uses the 'textstat' library to calculate objective readability scores.
    This is a custom tool enhanced with a specialized library.
    Returns a JSON string with the Flesch Reading Ease score and interpretation.
    """
    if not text.strip():
        return json.dumps({"score": 0, "level": "unassessable"})
    
    score = textstat.flesch_reading_ease(text)
    level = "Very Easy"
    if score < 30:
        level = "Very Confusing (College Graduate)"
    elif score < 60:
        level = "Difficult"
    elif score < 80:
        level = "Fairly Easy"
    return json.dumps({"flesch_reading_ease_score": score, "level": level})

def check_professionalism(text: str) -> str:
    """
    Checks for unprofessional content like profanity, slang, or excessive formatting.
    This is fast and rule-based, perfect for a custom tool.
    Returns a JSON string with a list of professionalism issues.
    """
    issues = []
    # Check for profanity
    if re.search(r'\b(fuck|shit|damn|bitch|asshole)\b', text, re.IGNORECASE):
        issues.append("Inappropriate or profane language was found.")
    # Check for "shouting" (excessive capitalization)
    if re.search(r'\b[A-Z]{4,}\b', text) and sum(1 for c in text if c.isupper()) / len(text) > 0.3:
        issues.append("Excessive capitalization is used, which appears unprofessional.")
    # Check for excessive punctuation
    if re.search(r'[!?@#$%^&*()]{4,}', text):
        issues.append("Excessive punctuation or symbols are used.")
    return json.dumps({"issues": issues})


# ==============================================================================
#  THE REFACTORED AGENT CLASS
# ==============================================================================

class ContentQualityAgent:
    def __init__(self, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.setup_agent()

    def setup_agent(self):
        """Setup an agent with a mix of built-in and custom tools."""
        
        # 1. Define the tools list
        tools = [
            # --- BUILT-IN TOOL ---
            TavilySearchResults(
                name="FactCheckSearch",
                max_results=3,
                description="Use this tool to search the web to verify factual claims, check for misinformation, or find context on a topic."
            ),
            # --- UPGRADED & CUSTOM TOOLS ---
            Tool(
                name="GrammarAndSpellingCheck",
                func=check_grammar_with_llm,
                description="Use this to check a piece of text for spelling and grammatical errors."
            ),
            Tool(
                name="ReadabilityCheck",
                func=check_readability_with_textstat,
                description="Use this to get the readability score and complexity level of a text."
            ),
            Tool(
                name="ProfessionalismCheck",
                func=check_professionalism,
                description="Use this to check for unprofessional language, like profanity, excessive capitalization, or slang."
            ),
            Tool(
                name="LengthAndStructureCheck",
                func=check_length_and_structure,
                description="Use this to get basic statistics like word count and paragraph structure."
            ),
        ]
        
        # 2. Create the agent's prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert Quality Assurance assistant. Your goal is to provide a complete and accurate quality report on the user's text. Use your tools to check all aspects of the content. First, form a plan, then execute your plan by calling the necessary tools. Finally, compile all the results into a comprehensive final answer summarizing your findings."),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        # 3. Create the agent
        agent = create_openai_functions_agent(self.llm, tools, prompt)
        
        # 4. Create the Agent Executor to run the agent
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    def validate(self, text_to_validate: str) -> Dict[str, Any]:
        """Runs the agent to perform a full quality analysis."""
        prompt = f"""
        Please provide a comprehensive quality analysis of the following text.
        Your final answer should be a summary of all findings.

        Text to Analyze:
        ---
        {text_to_validate}
        ---
        """
        return self.agent_executor.invoke({"input": prompt})

# ==============================================================================
#  MAIN EXECUTION BLOCK
# ==============================================================================

def main():
    if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
        print("❌ Error: OPENAI_API_KEY and TAVILY_API_KEY must be set in your .env file.")
        return

    print("🚀 Initializing Content Quality Agent...")
    validator = ContentQualityAgent()

    sample_text = """
    BREAKING NEWS!!! The goverment has SECRETLEY been using 5G towers to controll our minds and make us buy more STUFF we dont need!!! This is CIENTIFICALY proven by my frend who works at Area 51 and he told me that aliens are actualy running Facebook and Google to steal our THOUGHTS!!! Mark Zuckerberg is definatley an alien REPTILIAN who eats babys for breakfast FACT!!! Wake up SHEEPLE the truth is RIGHT THERE!!!! #conspiracy #truth #wakeup !@#$%^&*()
    """
    
    print("\n🔍 Running Full Analysis...")
    print("=" * 60)
    
    result = validator.validate(sample_text)
    
    print("\n✅ FINAL AGENT REPORT:")
    print("-" * 60)
    print(result['output'])

if __name__ == "__main__":
    main()

In [ ]:
import json
import re
import os
from typing import Dict, Any, Union

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import textstat

load_dotenv()

# ==============================================================================
#  TOOL LOGIC FUNCTIONS
# ==============================================================================

def check_length_and_structure(text: str) -> str:
    """Analyzes content length and structure (word count, paragraphs)."""
    # This function is now focused on a single piece of text
    words = text.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    issues = []
    if word_count < 15:
        issues.append(f"Content is very short with only {word_count} words.")
    if word_count > 150 and len(paragraphs) == 1:
        issues.append("Long content is not broken into paragraphs, making it hard to read.")
    if len(sentences) < 2 and word_count > 20:
        issues.append("Content consists of a single long sentence; consider breaking it up.")
    return json.dumps({"word_count": word_count, "sentences": len(sentences), "paragraphs": len(paragraphs), "issues": issues})

def check_grammar_with_llm(text: str) -> str:
    """Uses a powerful LLM to check for grammar and spelling errors."""
    print("🧠 Using LLM to check grammar...")
    checker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an expert proofreader. Analyze the following text for grammar and spelling mistakes.
    Respond with a JSON object: {{"errors": ["list of concise error descriptions"]}}. If no errors, return an empty list.
    Text: "{text}"
    """
    response = checker_llm.invoke(prompt)
    return response.content

def check_readability_with_textstat(text: str) -> str:
    """Uses the 'textstat' library to calculate objective readability scores."""
    if not text.strip():
        return json.dumps({"score": 0, "level": "unassessable"})
    score = textstat.flesch_reading_ease(text)
    level = "Very Easy"
    if score < 30: level = "Very Confusing (College Graduate)"
    elif score < 60: level = "Difficult"
    elif score < 80: level = "Fairly Easy"
    return json.dumps({"flesch_reading_ease_score": score, "level": level})

def check_professionalism(text: str) -> str:
    """Checks for unprofessional content like profanity, slang, or excessive formatting."""
    issues = []
    if re.search(r'\b(fuck|shit|damn|bitch|asshole)\b', text, re.IGNORECASE):
        issues.append("Inappropriate or profane language was found.")
    if re.search(r'\b[A-Z]{4,}\b', text) and sum(1 for c in text if c.isupper()) / len(text) > 0.3:
        issues.append("Excessive capitalization is used, which appears unprofessional.")
    if re.search(r'[!?@#$%^&*()]{4,}', text):
        issues.append("Excessive punctuation or symbols are used.")
    return json.dumps({"issues": issues})

def check_redundancy(text: str) -> str:
    """Analyzes the text for repetitive sentences and overused words."""
    sentences = [s.lower().strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    issues = []
    if len(sentences) > 2:
        redundancy_ratio = len(set(sentences)) / len(sentences)
        if redundancy_ratio < 0.5:
            issues.append(f"High sentence redundancy detected. {100 - redundancy_ratio*100:.0f}% of sentences are repetitive.")
    return json.dumps({"redundancy_issues": issues})

def check_itext_payload_faithfulness(payload_str: str) -> str:
    """
    Analyzes an iText JSON payload. It checks for consistency between the message and data,
    and evaluates the faithfulness and relevance between the 'content', 'simplified', and 'elaborated' versions.
    """
    print("🧠 Using LLM to analyze iText payload...")
    try:
        payload = json.loads(payload_str)
        message = payload.get("message", "")
        content_data = payload.get("data", [{}])[0].get("data", {})
        original = content_data.get("content", "")
        simplified = content_data.get("simplified", "")
        elaborated = content_data.get("elaborated", "")
    except (json.JSONDecodeError, IndexError, KeyError) as e:
        return json.dumps({"error": f"Invalid iText payload structure: {e}"})

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an expert iText payload evaluator. Analyze the following payload:
    Message: "{message}"
    Original Content: "{original}"
    Simplified Content: "{simplified}"
    Elaborated Content: "{elaborated}"

    Evaluate two things:
    1.  **Consistency**: Does the content seem to match the message (e.g., a "success" message with valid content)?
    2.  **Faithfulness & Relevance**: Compare the three text versions. Do the simplified/elaborated versions maintain the original meaning without adding incorrect info?

    Return ONLY a JSON object with your findings, like this:
    {{
        "consistency_check": "<Brief summary of consistency>",
        "faithfulness_score": <int 1-10>,
        "relevance_score": <int 1-10>,
        "comparison_summary": "<Brief summary of how the versions differ>"
    }}
    """
    response = llm.invoke(prompt)
    return response.content

# ==============================================================================
#  AGENT CLASS
# ==============================================================================

class ContentQualityAgent:
    def __init__(self, model="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.setup_agent()

    def setup_agent(self):
        tools = [
            Tool(name="iTextPayloadFaithfulnessCheck", func=check_itext_payload_faithfulness, description="Use this ONLY for a JSON input representing an iText payload. It checks faithfulness between content versions."),
            Tool(name="GrammarAndSpellingCheck", func=check_grammar_with_llm, description="Use to check a piece of text for spelling and grammatical errors."),
            Tool(name="ReadabilityCheck", func=check_readability_with_textstat, description="Use to get the readability score of a piece of text."),
            Tool(name="ProfessionalismCheck", func=check_professionalism, description="Use to check a piece of text for unprofessional language."),
            Tool(name="LengthAndStructureCheck", func=check_length_and_structure, description="Use to get word count and structure of a piece of text."),
            Tool(name="RedundancyCheck", func=check_redundancy, description="Use to check a piece of text for repetitive sentences."),
            TavilySearchResults(name="FactCheckSearch", max_results=3, description="Use to verify factual claims in a piece of text.")
        ]
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert Quality Assurance assistant. Your goal is to provide a complete quality report.

            You will receive one of two types of input:
            1.  A plain text paragraph.
            2.  A JSON string representing an "iText payload".

            **Your Plan:**
            - If the input is an iText payload (JSON), you MUST use the `iTextPayloadFaithfulnessCheck` tool on the entire JSON. You should ALSO run standard checks (grammar, professionalism, etc.) on the 'content' field within the payload.
            - If the input is plain text, just run the standard checks on it.
            
            After executing your plan, compile all results into the final JSON report.

            **FINAL ANSWER FORMATTING INSTRUCTIONS:**
            Your final answer MUST be a single JSON object with the specified structure.
            {{
                "overall_score": <An integer score from 1-10>,
                "category_scores": {{
                    "grammar_and_spelling": <Score 1-10>,
                    "readability": <Score 1-10>,
                    "professionalism_and_tone": <Score 1-10>,
                    "factual_accuracy": <Score 1-10. Default to 10 if no claims to check.>,
                    "redundancy": <Score 1-10>,
                    "faithfulness": <Score 1-10. Only applies to iText. Default to 10 for plain text.>
                }},
                "summary": "<A natural language summary of key issues and suggestions for improvement.>"
            }}
            """),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        agent = create_openai_functions_agent(self.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    async def validate_async(self, input_data: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        """Runs the agent asynchronously on either a string or a dictionary."""
        
        if isinstance(input_data, dict):
            # It's an iText payload, serialize it to a string for the agent
            input_str = json.dumps(input_data, indent=2)
            analysis_subject = "the following iText payload"
        else:
            # It's a plain paragraph
            input_str = input_data
            analysis_subject = "the following text"
            
        prompt = f"Please provide a comprehensive quality analysis of {analysis_subject}:\n---\n{input_str}\n---"
        
        result = {} 
        try:
            result = await self.agent_executor.ainvoke({"input": prompt})
            # Use regex to find the JSON object in case the LLM adds extra text
            json_match = re.search(r'\{.*\}', result['output'], re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                raise json.JSONDecodeError("No JSON object found in agent output.", result['output'], 0)
        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error during agent execution or parsing: {e}")
            return {"error": "Failed to generate a valid report.", "raw_output": result.get('output', 'No output was generated.')}

# ==============================================================================
#  MAIN EXECUTION BLOCK
# ==============================================================================
async def main():
    if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
        print("❌ Error: API keys must be set in your .env file.")
        return

    print("🚀 Initializing Content Quality Agent...")
    validator = ContentQualityAgent()
    
    # # --- CASE 1: Plain Paragraph ---
    # paragraph_to_analyze = """
    # Machine learning is the most revolutionary technology ever created and will solve all human problems. However, machine learning is completely useless and will never work properly. AI systems are incredibly intelligent. AI is actually very stupid. Neural networks are perfect. Neural networks are terrible.
    # """
    # print("\n\n🔍 Running analysis on Plain Paragraph...")
    # print("=" * 60)
    # report1 = await validator.validate_async(paragraph_to_analyze)
    # print_report(report1)

    # --- CASE 2: iText Payload ---
    itext_payload = {
        "message": "Successfully generated iText block data.",
        "data": [
            {
                "type": "iText",
                "data": {
                    "content": "The sun is a star at the center of the Solar System. It is a nearly perfect ball of hot plasma. The Sun's diameter is about 109 times that of Earth.",
                    "simplified": "The sun is a big star in the middle of our solar system.",
                    "elaborated": "The Sun, a G-type main-sequence star (G2V), is the focal point of our Solar System. Comprising over 99.8% of the system's mass, it is an immense sphere of incandescent plasma, with a diameter of approximately 1.39 million kilometers, roughly 109 times that of our planet, which is a big lie."
                }
            }
        ]
    }
    print("\n\n🔍 Running analysis on iText Payload...")
    print("=" * 60)
    report2 = await validator.validate_async(itext_payload)
    print_report(report2)


def print_report(report: Dict[str, Any]):
    """Helper function to neatly print the agent's report."""
    print("\n✅ FINAL AGENT REPORT:")
    print("-" * 60)
    
    if "error" in report:
        print(f"An error occurred: {report['error']}")
        print(f"Raw Output: {report.get('raw_output')}")
    else:
        print(f"🏆 Overall Quality Score: {report.get('overall_score', 'N/A')} / 10")
        print("\n--- Category Breakdown ---")
        for category, score in report.get('category_scores', {}).items():
            category_name = category.replace('_', ' ').title()
            print(f"   - {category_name}: {score}/10")
        
        print("\n--- Summary & Suggestions ---")
        print(report.get('summary', 'No summary provided.'))
    
    print("-" * 60)

# Run the main async function
if __name__ == "__main__":
    await main()

In [ ]:
import json
import re
import os
from typing import Dict, Any, List, Union
import asyncio

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import textstat

load_dotenv()

# ==============================================================================
# TOOL LOGIC FUNCTIONS (Made async for concurrent execution)
# ==============================================================================

async def check_length_and_structure(text: str) -> str:
    words = text.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    issues = []
    if word_count < 15:
        issues.append(f"Content is very short with only {word_count} words.")
    if word_count > 150 and len(paragraphs) == 1:
        issues.append("Long content is not broken into paragraphs, making it hard to read.")
    if len(sentences) < 2 and word_count > 20:
        issues.append("Content consists of a single long sentence; consider breaking it up.")
    return json.dumps({"word_count": word_count, "sentences": len(sentences), "paragraphs": len(paragraphs), "issues": issues})


async def check_grammar_with_llm(text: str) -> str:
    print("🧠 Using LLM to check grammar...")
    # Use a cheaper/faster model for simple tasks
    checker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an expert proofreader. Analyze the following text for grammar and spelling mistakes.
    Respond ONLY with JSON: {{"errors": ["list of concise error descriptions"]}}. If no errors, return an empty list.
    Text: "{text}"
    """
    try:
        response = await checker_llm.ainvoke(prompt)
        return response.content
    except Exception as e:
        return json.dumps({"errors": [f"LLM call failed: {e}"]})


async def check_readability_with_textstat(text: str) -> str:
    if not text.strip():
        return json.dumps({"score": 0, "level": "unassessable"})
    score = textstat.flesch_reading_ease(text)
    level = "Very Easy"
    if score < 30: level = "Very Confusing (College Graduate)"
    elif score < 60: level = "Difficult"
    elif score < 80: level = "Fairly Easy"
    return json.dumps({"flesch_reading_ease_score": score, "level": level})


async def check_professionalism(text: str) -> str:
    issues = []
    if re.search(r'\b(fuck|shit|damn|bitch|asshole)\b', text, re.IGNORECASE):
        issues.append("Inappropriate or profane language was found.")
    # Adjusted regex to be more accurate for excessive caps
    if sum(1 for c in text if c.isupper()) / len(text.replace(" ", "")) > 0.4 and len(text) > 20:
         issues.append("Excessive capitalization is used, which appears unprofessional.")
    if re.search(r'[!?@#$%^&*()_+=]{4,}', text):
        issues.append("Excessive punctuation or symbols are used.")
    return json.dumps({"issues": issues})


async def check_redundancy(text: str) -> str:
    sentences = [s.lower().strip() for s in re.split(r'[.!?]+', text) if len(s.strip().split()) > 3]
    issues = []
    if len(sentences) > 2:
        # Using a more robust check for redundancy
        unique_sentences = set(sentences)
        redundancy_ratio = len(unique_sentences) / len(sentences)
        if redundancy_ratio < 0.6:
            issues.append(f"High sentence redundancy detected. {100 - redundancy_ratio*100:.0f}% of sentences seem repetitive.")
    return json.dumps({"redundancy_issues": issues})


async def check_faithfulness_and_relevance(payload: Dict[str, Any]) -> str:
    """Compare original vs simplified and elaborated text."""
    print("🧠 Using LLM to check faithfulness and relevance...")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    content = payload.get("content", "")
    simplified = payload.get("simplified", "")
    elaborated = payload.get("elaborated", "")

    prompt = f"""
    You are an expert text evaluator. Compare these three versions of content:
    [Original] {content}
    [Simplified] {simplified}
    [Elaborated] {elaborated}

    Evaluate on a scale of 1-10 (1=bad, 10=excellent) and provide a summary.
    1. Faithfulness — Do simplified/elaborated keep the original meaning?
    2. Relevance — Are they contextually correct and not adding wrong info?
    3. Accuracy — Are there factual or conceptual mistakes?

    Return ONLY JSON with this exact structure:
    {{
        "faithfulness": <int>,
        "relevance": <int>,
        "accuracy": <int>,
        "summary": "<short explanation of issues or differences>"
    }}
    """
    try:
        response = await llm.ainvoke(prompt)
        return response.content
    except Exception as e:
        return json.dumps({
            "faithfulness": 1, "relevance": 1, "accuracy": 1,
            "summary": f"LLM call failed: {e}"
        })

# ==============================================================================
# VALIDATOR CLASS (Replaces the Agent)
# ==============================================================================

class ContentQualityValidator:
    def __init__(self, model="gpt-4o", temperature=0.1):
        # Use a powerful model for the final synthesis step
        self.synthesis_llm = ChatOpenAI(model=model, temperature=temperature)

    async def validate_async(self, input_data: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        """
        Runs all checks concurrently and synthesizes a final report.
        """
        tasks = []
        is_itext = isinstance(input_data, dict) and "content" in input_data

        # Use the 'content' field for standard checks if available, otherwise use the whole input
        text_to_check = input_data.get("content") if is_itext else input_data

        # --- 1. GATHER DATA FROM ALL CHECKS ---
        tasks.append(check_grammar_with_llm(text_to_check))
        tasks.append(check_readability_with_textstat(text_to_check))
        tasks.append(check_professionalism(text_to_check))
        tasks.append(check_redundancy(text_to_check))
        tasks.append(check_length_and_structure(text_to_check))
        
        if is_itext:
            tasks.append(check_faithfulness_and_relevance(input_data))

        # Run all tasks in parallel
        results = await asyncio.gather(*tasks)

        # --- 2. PREPARE CONTEXT FOR SYNTHESIS ---
        context = {
            "grammar_and_spelling_report": json.loads(results[0]),
            "readability_report": json.loads(results[1]),
            "professionalism_report": json.loads(results[2]),
            "redundancy_report": json.loads(results[3]),
            "length_and_structure_report": json.loads(results[4]),
        }
        if is_itext:
            context["faithfulness_and_relevance_report"] = json.loads(results[5])
        
        # --- 3. SYNTHESIZE THE FINAL REPORT ---
        print("🧠 Synthesizing final report...")
        synthesis_prompt = f"""
        You are an intelligent QA Analyst. Based on the following raw data from various quality checks,
        generate a final JSON report.

        Your task is to convert the raw data into category scores from 1-10 (1=very poor, 10=excellent)
        and write a concise summary with actionable improvement suggestions.

        Raw Data:
        {json.dumps(context, indent=2)}

        - For `grammar_and_spelling`, if there are errors, score is low. No errors = 10.
        - For `readability`, a "Very Easy" or "Fairly Easy" level is high-scoring (8-10). "Difficult" is 4-6. "Very Confusing" is 1-3.
        - For `professionalism_and_tone`, any issues found should result in a low score. No issues = 10.
        - For `redundancy`, any issues found means a lower score. No issues = 10.
        - `factual_accuracy` is 10 by default unless faithfulness/relevance checks find issues.
        - `faithfulness` and `relevance` scores should be taken directly from the report if present, otherwise set them to 10.

        FINAL ANSWER must be ONLY the JSON object with the following structure:
        {{
            "overall_score": <int 1-10>,
            "category_scores": {{
                "grammar_and_spelling": <int 1-10>,
                "readability": <int 1-10>,
                "professionalism_and_tone": <int 1-10>,
                "factual_accuracy": <int 1-10>,
                "redundancy": <int 1-10>,
                "faithfulness": <int 1-10>,
                "relevance": <int 1-10>
            }},
            "summary": "<summary of findings and concrete improvement suggestions>"
        }}
        """
        
        try:
            final_response = await self.synthesis_llm.ainvoke(synthesis_prompt)
            # Find the JSON object in the response, in case the LLM adds extra text
            json_match = re.search(r'\{.*\}', final_response.content, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                raise json.JSONDecodeError("No JSON object found in the final response.", final_response.content, 0)
        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error parsing final JSON response: {e}")
            return {"error": "Invalid or failed final JSON synthesis", "raw_output": final_response.content}


# ==============================================================================
# MAIN
# ==============================================================================
async def main():
    validator = ContentQualityValidator()

    # # --- CASE 1: iText JSON payload ---
    # itext_payload = {
    #     "message": "Successfully generated iText block data.",
    #     "data": [
    #         {
    #             "type": "iText",
    #             "data": {
    #                 "content": "Foundations of Deep Learning serve as the bedrock for modern AI, encompassing neural networks, backpropagation, and activation functions. These principles enable machines to learn from vast datasets.",
    #                 "simplified": "Deep learning is a type of machine learning that uses artificial neural networks to learn from data. It's the foundation of today's AI.",
    #                 "elaborated": "The foundations of deep learning encompass a range of critical concepts, starting with the artificial neuron, the basic computational unit. These are organized into layers to form deep neural networks. The learning process is facilitated by an algorithm called backpropagation, which adjusts the network's parameters to minimize error, and non-linear activation functions like ReLU, which allow the network to model complex patterns. This is all a lie and the sky is green."
    #             }
    #         }
    #     ]
    # }

    # print("\n🚀 Evaluating iText payload...")
    # itext_data = itext_payload["data"][0]["data"]
    # report = await validator.validate_async(itext_data)
    # print(json.dumps(report, indent=2))


    # # --- CASE 2: plain paragraph 
# Run main
if __name__ == "__main__":
    await (main())

In [4]:
import json
import re
import os
from typing import Dict, Any, Union

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import textstat

load_dotenv()

# ==============================================================================
#  TOOL LOGIC FUNCTIONS
# ==============================================================================

def check_length_and_structure(text: str) -> str:
    """Analyzes content length and structure (word count, paragraphs)."""
    words = text.split()
    word_count = len(words)
    # ... (rest of the function is unchanged)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    issues = []
    if word_count < 15:
        issues.append(f"Content is very short with only {word_count} words.")
    if word_count > 150 and len(paragraphs) == 1:
        issues.append("Long content is not broken into paragraphs, making it hard to read.")
    if len(sentences) < 2 and word_count > 20:
        issues.append("Content consists of a single long sentence; consider breaking it up.")
    return json.dumps({"word_count": word_count, "sentences": len(sentences), "paragraphs": len(paragraphs), "issues": issues})

def check_grammar_with_llm(text: str) -> str:
    """Uses a powerful LLM to check for grammar and spelling errors."""
    print("🧠 Using LLM to check grammar...")
    checker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an expert proofreader. Analyze the following text for grammar and spelling mistakes.
    Respond with a JSON object: {{"errors": ["list of concise error descriptions"]}}. If no errors, return an empty list.
    Text: "{text}"
    """
    response = checker_llm.invoke(prompt)
    return response.content

def check_readability_with_textstat(text: str) -> str:
    """Uses the 'textstat' library to calculate objective readability scores."""
    if not text.strip():
        return json.dumps({"score": 0, "level": "unassessable"})
    score = textstat.flesch_reading_ease(text)
    level = "Very Easy"
    if score < 30: level = "Very Confusing (College Graduate)"
    elif score < 60: level = "Difficult"
    elif score < 80: level = "Fairly Easy"
    return json.dumps({"flesch_reading_ease_score": score, "level": level})

def check_professionalism(text: str) -> str:
    """Checks for unprofessional content like profanity, slang, or excessive formatting."""
    issues = []
    if re.search(r'\b(fuck|shit|damn|bitch|asshole)\b', text, re.IGNORECASE):
        issues.append("Inappropriate or profane language was found.")
    if re.search(r'\b[A-Z]{4,}\b', text) and sum(1 for c in text if c.isupper()) / len(text) > 0.3:
        issues.append("Excessive capitalization is used, which appears unprofessional.")
    if re.search(r'[!?@#$%^&*()]{4,}', text):
        issues.append("Excessive punctuation or symbols are used.")
    return json.dumps({"issues": issues})

# This is now the "super-tool" for iText Payloads
def comprehensive_itext_checker(payload_str: str) -> str:
    """
    Performs a FULL analysis on an iText JSON payload. It checks faithfulness between versions
    AND runs grammar, readability, and professionalism checks on the content, simplified, and elaborated texts.
    """
    print("🔬 Performing comprehensive iText analysis...")
    try:
        payload = json.loads(payload_str)
        content_data = payload.get("data", [{}])[0].get("data", {})
        original = content_data.get("content", "")
        simplified = content_data.get("simplified", "")
        elaborated = content_data.get("elaborated", "")
    except (json.JSONDecodeError, IndexError, KeyError) as e:
        return json.dumps({"error": f"Invalid iText payload structure: {e}"})

    # --- 1. Perform Faithfulness Check with an LLM ---
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    faithfulness_prompt = f"""
    You are an expert iText payload evaluator. Compare the following three text versions.
    Evaluate their faithfulness and relevance on a scale of 1-10 and provide a summary.
    [Original]: {original}
    [Simplified]: {simplified}
    [Elaborated]: {elaborated}
    Return ONLY a JSON object with your findings.
    {{
        "faithfulness_score": <int 1-10>,
        "relevance_score": <int 1-10>,
        "comparison_summary": "<Brief summary of how the versions differ>"
    }}
    """
    faithfulness_report = json.loads(llm.invoke(faithfulness_prompt).content)

    # --- 2. Perform Standard Checks on Each Text Field ---
    texts_to_check = {
        "original": original,
        "simplified": simplified,
        "elaborated": elaborated
    }
    detailed_reports = {}
    for key, text in texts_to_check.items():
        if text: # Only run checks if the text exists
            detailed_reports[key] = {
                "grammar_report": json.loads(check_grammar_with_llm(text)),
                "readability_report": json.loads(check_readability_with_textstat(text)),
                "professionalism_report": json.loads(check_professionalism(text)),
            }

    # --- 3. Combine All Results ---
    final_combined_report = {
        "faithfulness_analysis": faithfulness_report,
        "detailed_text_analysis": detailed_reports
    }

    return json.dumps(final_combined_report, indent=2)

# ==============================================================================
#  AGENT CLASS
# ==============================================================================

class ContentQualityAgent:
    def __init__(self, model="gpt-4o", temperature=0): # Use a powerful model for the final reasoning
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.setup_agent()

    def setup_agent(self):
        # Note: We only provide the super-tool for iText now.
        # The agent's job is simpler: pick the right tool for the job.
        tools = [
            Tool(
                name="ComprehensiveITextChecker",
                func=comprehensive_itext_checker,
                description="Use this ONLY for a JSON input representing an iText payload. This tool performs a complete analysis."
            ),
            Tool(name="GrammarAndSpellingCheck", func=check_grammar_with_llm, description="Use to check a piece of plain text for spelling and grammatical errors."),
            Tool(name="ReadabilityCheck", func=check_readability_with_textstat, description="Use to get the readability score of a piece of plain text."),
            Tool(name="ProfessionalismCheck", func=check_professionalism, description="Use to check a piece of plain text for unprofessional language."),
        ]
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert Quality Assurance assistant. Your job is to create a final, summarized quality report in JSON format.

            **Your Plan:**
            - **If the input is an iText payload (JSON string):** Use the `ComprehensiveITextChecker` tool. This one tool will give you all the data you need.
            - **If the input is plain text:** Use the standard tools like `GrammarAndSpellingCheck` and `ReadabilityCheck` to gather data.
            
            After getting the tool results, analyze them and create the final summary report as requested.

            **FINAL ANSWER FORMATTING INSTRUCTIONS:**
            Your final answer MUST be a single JSON object with the specified structure.
            {{
                "overall_score": <An integer score from 1-10>,
                "category_scores": {{
                    "grammar_and_spelling": <Score 1-10>,
                    "readability": <Score 1-10>,
                    "professionalism_and_tone": <Score 1-10>,
                    "faithfulness": <Score 1-10. Default to 10 for plain text.>
                }},
                "summary": "<A natural language summary of key issues and suggestions for improvement.>"
            }}
            """),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        agent = create_openai_functions_agent(self.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    async def validate_async(self, input_data: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        """Runs the agent asynchronously on either a string or a dictionary."""
        
        if isinstance(input_data, dict):
            input_str = json.dumps(input_data)
            analysis_subject = "the following iText payload"
        else:
            input_str = input_data
            analysis_subject = "the following text"
            
        prompt = f"Please provide a comprehensive quality analysis of {analysis_subject}:\n---\n{input_str}\n---"
        
        result = {} 
        try:
            result = await self.agent_executor.ainvoke({"input": prompt})
            json_match = re.search(r'\{.*\}', result['output'], re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                raise json.JSONDecodeError("No JSON object found in agent output.", result['output'], 0)
        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error during agent execution or parsing: {e}")
            return {"error": "Failed to generate a valid report.", "raw_output": result.get('output', 'No output was generated.')}

# ==============================================================================
#  MAIN EXECUTION BLOCK
# ==============================================================================
def print_report(report: Dict[str, Any], title: str):
    """Helper function to neatly print the agent's report."""
    print(f"\n✅ FINAL AGENT REPORT: {title}")
    print("-" * 60)
    
    if "error" in report:
        print(f"An error occurred: {report['error']}")
        print(f"Raw Output: {report.get('raw_output')}")
    else:
        print(f"🏆 Overall Quality Score: {report.get('overall_score', 'N/A')} / 10")
        print("\n--- Category Breakdown ---")
        for category, score in report.get('category_scores', {}).items():
            category_name = category.replace('_', ' ').title()
            print(f"   - {category_name}: {score}/10")
        
        print("\n--- Summary & Suggestions ---")
        print(report.get('summary', 'No summary provided.'))
    
    print("-" * 60)

async def main():
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ Error: OPENAI_API_KEY must be set in your .env file.")
        return

    print("🚀 Initializing Content Quality Agent...")
    validator = ContentQualityAgent()
    
    # --- CASE 1: Plain Paragraph ---
    paragraph_to_analyze = """
Identifying Market Needs

Identifying market needs is a critical step in the ideation and concept development phase of launching a startup. Understanding what potential customers desire or require enables entrepreneurs to tailor their products or services effectively. This process involves conducting thorough market research, analyzing trends, and gathering insights directly from the target audience. Entrepreneurs must consider various factors, including demographics, psychographics, and behavioral patterns, to accurately identify market needs. By leveraging qualitative and quantitative research methods, such as surveys, interviews, and focus groups, startups can gain a comprehensive understanding of their market landscape. This foundational knowledge not only informs product development but also enhances marketing strategies, ensuring that the startup addresses genuine customer pain points.
"""
    

    print("=" * 60)
    report1 = await validator.validate_async(paragraph_to_analyze)
    print_report(report1, "Plain Paragraph Analysis")

    # --- CASE 2: iText Payload ---
    # itext_payload = {
    #     "message": "Generate the successful content regarding typescript.",
    #     "data": [{"type": "iText", "data": {
    #         "content": "Foundations of Deep Learning serve as the bedrock for understanding neural networks...",
    #         "simplified": "Deep learning is a type of machine learning that uses networks of nodes...",
    #         "elaborated": "The foundations of deep learning encompass a range of principles and practices..."
    #     }}]
    # }
    # print("\n\n🔍 Running analysis on iText Payload...")
    # print("=" * 60)
    # report2 = await validator.validate_async(itext_payload)
    # print_report(report2, "iText Payload Analysis")

if __name__ == "__main__":

    await(main())

🚀 Initializing Content Quality Agent...


> Entering new AgentExecutor chain...

Invoking: `GrammarAndSpellingCheck` with `Identifying Market Needs

Identifying market needs is a critical step in the ideation and concept development phase of launching a startup. Understanding what potential customers desire or require enables entrepreneurs to tailor their products or services effectively. This process involves conducting thorough market research, analyzing trends, and gathering insights directly from the target audience. Entrepreneurs must consider various factors, including demographics, psychographics, and behavioral patterns, to accurately identify market needs. By leveraging qualitative and quantitative research methods, such as surveys, interviews, and focus groups, startups can gain a comprehensive understanding of their market landscape. This foundational knowledge not only informs product development but also enhances marketing strategies, ensuring that the startup addresses ge

In [3]:
import json
import re
import os
from typing import Dict, Any, Union

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import textstat

load_dotenv()

# ==============================================================================
#  TOOL LOGIC FUNCTIONS (Unchanged)
# ==============================================================================

def check_length_and_structure(text: str) -> str:
    """Analyzes content length and structure (word count, paragraphs)."""
    words = text.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    issues = []
    if word_count < 15:
        issues.append(f"Content is very short with only {word_count} words.")
    if word_count > 150 and len(paragraphs) == 1:
        issues.append("Long content is not broken into paragraphs, making it hard to read.")
    if len(sentences) < 2 and word_count > 20:
        issues.append("Content consists of a single long sentence; consider breaking it up.")
    return json.dumps({"word_count": word_count, "sentences": len(sentences), "paragraphs": len(paragraphs), "issues": issues})

def check_grammar_with_llm(text: str) -> str:
    """Uses a powerful LLM to check for grammar and spelling errors."""
    print("🧠 Using LLM to check grammar...")
    checker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an expert proofreader. Analyze the following text for grammar and spelling mistakes.
    Respond with a JSON object: {{"errors": ["list of concise error descriptions"]}}. If no errors, return an empty list.
    Text: "{text}"
    """
    response = checker_llm.invoke(prompt)
    return response.content

def check_readability_with_textstat(text: str) -> str:
    """Uses the 'textstat' library to calculate objective readability scores."""
    if not text.strip():
        return json.dumps({"score": 0, "level": "unassessable"})
    score = textstat.flesch_reading_ease(text)
    level = "Very Easy"
    if score < 30: level = "Very Confusing (College Graduate)"
    elif score < 60: level = "Difficult"
    elif score < 80: level = "Fairly Easy"
    return json.dumps({"flesch_reading_ease_score": score, "level": level})

def check_professionalism(text: str) -> str:
    """Checks for unprofessional content like profanity, slang, or excessive formatting."""
    issues = []
    if re.search(r'\b(fuck|shit|damn|bitch|asshole)\b', text, re.IGNORECASE):
        issues.append("Inappropriate or profane language was found.")
    if re.search(r'\b[A-Z]{4,}\b', text) and sum(1 for c in text if c.isupper()) / len(text) > 0.3:
        issues.append("Excessive capitalization is used, which appears unprofessional.")
    if re.search(r'[!?@#$%^&*()]{4,}', text):
        issues.append("Excessive punctuation or symbols are used.")
    return json.dumps({"issues": issues})

def comprehensive_itext_checker(payload_str: str) -> str:
    """
    Performs a FULL analysis on an iText JSON payload, including all sub-checks.
    """
    print("🔬 Performing comprehensive iText analysis...")
    # This function's logic remains the same as before.
    try:
        payload = json.loads(payload_str)
        content_data = payload.get("data", [{}])[0].get("data", {})
        original = content_data.get("content", "")
        simplified = content_data.get("simplified", "")
        elaborated = content_data.get("elaborated", "")
    except (json.JSONDecodeError, IndexError, KeyError) as e:
        return json.dumps({"error": f"Invalid iText payload structure: {e}"})

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    faithfulness_prompt = f"""
    You are an expert iText payload evaluator. Compare the following three text versions.
    Evaluate their faithfulness and relevance on a scale of 1-10 and provide a summary.
    [Original]: {original}
    [Simplified]: {simplified}
    [Elaborated]: {elaborated}
    Return ONLY a JSON object with your findings.
    {{
        "faithfulness_score": <int 1-10>,
        "relevance_score": <int 1-10>,
        "comparison_summary": "<Brief summary of how the versions differ>"
    }}
    """
    faithfulness_report = json.loads(llm.invoke(faithfulness_prompt).content)
    texts_to_check = {"original": original, "simplified": simplified, "elaborated": elaborated}
    detailed_reports = {}
    for key, text in texts_to_check.items():
        if text:
            detailed_reports[key] = {
                "grammar_report": json.loads(check_grammar_with_llm(text)),
                "readability_report": json.loads(check_readability_with_textstat(text)),
                "professionalism_report": json.loads(check_professionalism(text)),
            }
    final_combined_report = {
        "faithfulness_analysis": faithfulness_report,
        "detailed_text_analysis": detailed_reports
    }
    return json.dumps(final_combined_report, indent=2)

# ==============================================================================
#  AGENT CLASS
# ==============================================================================

class ContentQualityAgent:
    def __init__(self, model="gpt-4o", temperature=0):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.setup_agent()

    def setup_agent(self):
        tools = [
            Tool(name="ComprehensiveITextChecker", func=comprehensive_itext_checker, description="Use this ONLY for a JSON input representing an iText payload. This tool performs a complete analysis."),
            Tool(name="GrammarAndSpellingCheck", func=check_grammar_with_llm, description="Use to check a piece of plain text for spelling and grammatical errors."),
            Tool(name="ReadabilityCheck", func=check_readability_with_textstat, description="Use to get the readability score of a piece of plain text."),
            Tool(name="ProfessionalismCheck", func=check_professionalism, description="Use to check a piece of plain text for unprofessional language."),
            TavilySearchResults(name="FactCheckSearch", max_results=2, description="Use to verify factual claims, check for misinformation, or find context for a piece of plain text.")
        ]
        
        # --- MODIFIED PROMPT ---
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert Quality Assurance assistant. Your job is to create a final, summarized quality report in JSON format.

            **Your Plan:**
            - **If the input is an iText payload (JSON string):** Use the `ComprehensiveITextChecker` tool.
            - **If the input is plain text:** Use tools like `GrammarAndSpellingCheck`, `ProfessionalismCheck`, and `FactCheckSearch` to gather data.
            
            After getting the tool results, analyze them and create the final summary report.

            **FINAL ANSWER FORMATTING INSTRUCTIONS:**
            Your final answer MUST be a single JSON object with the following structure.
            {{
                "overall_score": <An integer score from 1-10 based on all factors>,
                "category_scores": {{
                    "grammar_and_spelling": <Score 1-10. 10 is perfect.>,
                    "readability": <Score 1-10. Higher Flesch score = higher rating.>,
                    "professionalism_and_tone": <Score 1-10. Unprofessional content gets a very low score.>,
                    "factual_accuracy": <Score 1-10. Misinformation must get a score of 1. If no claims to check, score 10.>,
                    "faithfulness": <Score 1-10. Default to 10 for plain text.>
                }},
                "specific_mistakes": [
                    "<A list of specific errors found, e.g., 'Spelling error: `goverment` should be `government`'>",
                    "<e.g., 'Factual error: The claim about 5G is unsubstantiated.'>"
                ],
                "improvement_suggestions": [
                    "<A list of actionable suggestions, e.g., 'Proofread the text to correct all spelling mistakes.'>",
                    "<e.g., 'Remove excessive capitalization and punctuation to appear more professional.'>"
                ],
                "overall_summary": "<A concise, natural language summary of the findings.>"
            }}
            """),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        agent = create_openai_functions_agent(self.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    async def validate_async(self, input_data: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        """Runs the agent asynchronously on either a string or a dictionary."""
        if isinstance(input_data, dict):
            input_str = json.dumps(input_data)
            analysis_subject = "the following iText payload"
        else:
            input_str = input_data
            analysis_subject = "the following text"
            
        prompt = f"Please provide a comprehensive quality analysis of {analysis_subject}:\n---\n{input_str}\n---"
        
        result = {} 
        try:
            result = await self.agent_executor.ainvoke({"input": prompt})
            json_match = re.search(r'\{.*\}', result['output'], re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                raise json.JSONDecodeError("No JSON object found in agent output.", result['output'], 0)
        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Error during agent execution or parsing: {e}")
            return {"error": "Failed to generate a valid report.", "raw_output": result.get('output', 'No output was generated.')}

# ==============================================================================
#  MAIN EXECUTION BLOCK
# ==============================================================================
def print_report(report: Dict[str, Any], title: str):
    """Helper function to neatly print the agent's report."""
    print(f"\n✅ FINAL AGENT REPORT: {title}")
    print("-" * 60)
    
    if "error" in report:
        print(f"An error occurred: {report['error']}")
        print(f"Raw Output: {report.get('raw_output')}")
    else:
        print(f"🏆 Overall Quality Score: {report.get('overall_score', 'N/A')} / 10")
        print("\n--- Category Breakdown ---")
        for category, score in report.get('category_scores', {}).items():
            category_name = category.replace('_', ' ').title()
            print(f"   - {category_name}: {score}/10")
        
        print("\n--- Specific Mistakes Found ---")
        for mistake in report.get('specific_mistakes', ['None']):
            print(f"   - {mistake}")
            
        print("\n--- Improvement Suggestions ---")
        for suggestion in report.get('improvement_suggestions', ['None']):
            print(f"   - {suggestion}")

        print("\n--- Overall Summary ---")
        print(report.get('overall_summary', 'No summary provided.'))
    
    print("-" * 60)

async def main():
    if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
        print("❌ Error: API keys must be set in your .env file.")
        return

    print("🚀 Initializing Content Quality Agent...")
    validator = ContentQualityAgent()
    
    # --- Using your new paragraph as the input ---
    paragraph_to_analyze = """This fucking research paper is complete bullshit and the authors don't know what the hell they're talking about. The damn methodology is shit and their conclusions are fucking wrong. Anyone who believes this crap is a complete idiot and needs to get their head out of their ass. This is the worst piece of garbage I've ever read in my entire life."""  
    print("\n\n🔍 Running analysis on Plain Paragraph...")
    print("=" * 60)
    report1 = await validator.validate_async(paragraph_to_analyze)
    print_report(report1, "Conspiracy Paragraph Analysis")

if __name__ == "__main__":
    await (main())

c:\Users\karthikr\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Initializing Content Quality Agent...


C:\Users\karthikr\AppData\Local\Temp\ipykernel_48436\4238287781.py:129: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  TavilySearchResults(name="FactCheckSearch", max_results=2, description="Use to verify factual claims, check for misinformation, or find context for a piece of plain text.")




🔍 Running analysis on Plain Paragraph...


> Entering new AgentExecutor chain...

Invoking: `GrammarAndSpellingCheck` with `This fucking research paper is complete bullshit and the authors don't know what the hell they're talking about. The damn methodology is shit and their conclusions are fucking wrong. Anyone who believes this crap is a complete idiot and needs to get their head out of their ass. This is the worst piece of garbage I've ever read in my entire life.`


🧠 Using LLM to check grammar...
{"errors": ["Use of profanity may be inappropriate for academic or formal writing", "Informal language such as 'fucking', 'bullshit', 'damn', 'shit', 'crap', 'idiot', 'head out of their ass', and 'piece of garbage' is not suitable for a research paper"]}
Invoking: `ReadabilityCheck` with `This fucking research paper is complete bullshit and the authors don't know what the hell they're talking about. The damn methodology is shit and their conclusions are fucking wrong. Anyone who believe

SyntaxError: invalid non-printable character U+00A0 (3679537665.py, line 20)